In [1]:
import gradio as gr
from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [2]:
def convert_hf_ents_to_gradio(hf_ents):
    gradio_ents = []
    for hf_ent in hf_ents:
        gradio_ent = {"start" : hf_ent['start'], "end": hf_ent['end'], "entity": hf_ent['entity_group']}
        gradio_ents.append(gradio_ent)
    return gradio_ents

In [3]:
model_ckpt = '/Users/carolanderson/Dropbox/Website/blog_posts/food-ner/updated_roberta/checkpoint-740'
model_ckpt = "carolanderson/roberta-base-food-ner"

In [4]:
model = AutoModelForTokenClassification.from_pretrained(model_ckpt)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base", add_prefix_space=True)
# tokenizer = AutoTokenizer.from_pretrained("carolanderson/roberta-base-food-ner", add_prefix_space=True)  # produces warning that tokenizer does not support real words

In [9]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "Saute the onions in olive oil until browned."
example = "Add bourbon and sweet vermouth to the shaker."
example = "Salt the water and butter the bread."

ner_results = nlp(example, aggregation_strategy="first")
# ner_results = nlp(example)

print(ner_results)

[{'entity_group': 'FOOD', 'score': 0.5307181, 'word': ' water', 'start': 9, 'end': 14}, {'entity_group': 'FOOD', 'score': 0.99536836, 'word': ' bread.', 'start': 30, 'end': 36}]


In [7]:
def tag(text):
    hf_ents = nlp(text, aggregation_strategy="first")
    gradio_ents = convert_hf_ents_to_gradio(hf_ents)
    doc ={"text": text,
    "entities": gradio_ents}
    return doc

demo = gr.Interface(tag,
             gr.Textbox(placeholder="Enter text here..."), 
             gr.HighlightedText(),
              examples=[
        ["Saute the onions in olive oil until browned."],
        ["Add bourbon and sweet vermouth to the shaker."],
        ["Salt the water and butter the bread."],
        ["Add salt to the water and spread butter on the bread."]
    ])


demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/carolanderson/opt/anaconda3/envs/hf_space/lib/python3.9/site-packages/transformers/pipelines/token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
